In [ ]:
 !nvidia-smi

Sun Jul 24 07:39:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers , Model , Sequential
from tensorflow.keras.layers import Conv2D ,MaxPooling2D ,Flatten ,Dense, Dropout ,BatchNormalization,Input, AveragePooling2D 
import datetime
from tensorflow.keras.callbacks import ModelCheckpoint
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_train=x_train[10000:20000]
y_train = y_train[10000:20000]
y_train=tf.cast(y_train,dtype=tf.float32)

x_train = np.expand_dims(x_train,axis=-1)
x_train = np.repeat(x_train, 3, axis=-1)/255.
x_train=tf.image.resize(x_train, [224,224])
x_train = tf.cast(x_train > 0.5, dtype=tf.float32)

x_test=x_test[0:3000]
y_test=y_test[0:3000]
y_test=tf.cast(y_test,dtype=tf.float32)
x_test = np.expand_dims(x_test,axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)/255.
x_test=tf.image.resize(x_test, [224,224])
x_test = tf.cast(x_test > 0.5, dtype=tf.float32)

In [ ]:
x_train.shape,x_test.shape

(TensorShape([10000, 224, 224, 3]), TensorShape([3000, 224, 224, 3]))

In [ ]:
def _pairwise_distances(embaddings):
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2  , shape (batch_size, batch_size)
    dot_product = tf.matmul(embaddings,tf.transpose(embaddings))# n n
    square_norm = tf.linalg.diag_part(dot_product) # (n,)
    distances_sqared = tf.expand_dims(square_norm, 0) + tf.expand_dims(square_norm, 1) - 2*dot_product# n n
    distances= tf.sqrt(tf.math.maximum(distances_sqared,1e-16))# n n
    distances = tf.cast(distances,dtype=tf.float32)

    return distances

In [ ]:
def _get_anchor_positive_triplet_mask(labels):
    # Check if labels[i] == labels[j]
    labels_equal= tf.cast(tf.expand_dims(labels,0)==tf.expand_dims(labels,1),dtype=tf.float32)
    # Find index_i == index_j and than make them False
    equal_indexes=tf.eye(tf.shape(labels)[0])
    mask=labels_equal-equal_indexes
    return mask

In [ ]:
def _nonequal_negative_triplet_mask(labels):
    labels_equal= tf.cast(tf.expand_dims(labels,0)==tf.expand_dims(labels,1),dtype=tf.float32)
    # Check if labels[i] != labels[j]
    labels_nonequal = tf.ones((tf.shape(labels)[0],tf.shape(labels).shape[0]),dtype=tf.float32)-labels_equal
    return labels_nonequal

In [ ]:
def batch_hard_triplet_loss(y_true,y_pred,margin,All_positive=False,semi=False):
    labels= tf.squeeze(y_true,axis=-1)
    
    embeddings = y_pred

    pairwise_dist=_pairwise_distances(embeddings)
    positive_mask = _get_anchor_positive_triplet_mask(labels)
    positive_distance = tf.multiply(pairwise_dist, positive_mask)
    hardest_positive_distance = tf.reduce_max(positive_distance,axis=1,keepdims=True)
    if All_positive == True:
        loss = tf.reduce_mean(hardest_positive_distance)
        return loss
    # when taking maximum values away by a row in Matirx, minimum values are lower then zero. so we can choose minumum values by using function of minimum
    nonequal_mask = _nonequal_negative_triplet_mask(labels)
    distance_max = tf.math.reduce_max(pairwise_dist,1,keepdims=True)
    hardest_negative_distance = tf.math.reduce_min(
                    tf.cast(pairwise_dist-distance_max,dtype=tf.float32) * nonequal_mask,1,keepdims=True  
                    ) + distance_max
    if semi==False:
        loss=trplet_loss= tf.maximum( hardest_positive_distance - hardest_negative_distance + margin, 0.)
        loss= tf.reduce_mean(trplet_loss)
        return loss
    #semi hard
    negative_selection_mask = tf.cast(hardest_positive_distance < hardest_negative_distance,dtype=tf.float32)

    trplet_loss= tf.maximum(hardest_positive_distance - hardest_negative_distance + margin, 0.)*negative_selection_mask
    loss= tf.reduce_mean(trplet_loss)

    return loss

In [ ]:
def get_loss_function(margin=0.2,All_positive=False,semi=False):
  def loss(y_true,y_pred):
    return batch_hard_triplet_loss(y_true, y_pred,margin,All_positive,semi)
  return loss

In [ ]:
def l2_norm2d(x, pool_size = (3,3), strides = (1,1),
             padding = 'same', data_format=None):
    x = x ** 2
    output = AveragePooling2D(pool_size, strides,padding)(x)# (a^2+b^2+c^2...)/n
    output  = output*pool_size[0]*pool_size[1]# (a^2+b^2+c^2...)
    output = tf.sqrt(output)
    return output

In [ ]:
def inception(input,input_size, x3_in , x3_out ,x5_in , x5_out , pooling_option='max' ,strides=(1,1),pool_strides=(1,1), pool_output=None ):
   
    x1= Conv2D(filters = input_size,strides=(1,1), kernel_size= (1,1), padding= 'same')(input)
    x1=relu(x1)

    x3=Conv2D(filters = x3_in,strides=(1,1), kernel_size= (1,1), padding= 'same')(input)
    x3=Conv2D(filters = x3_out,strides=(strides) ,kernel_size= (3,3), padding= 'same')(x3)
    x3=relu(x3)

    x5=Conv2D(filters = x5_in,strides=(1,1), kernel_size= (1,1), padding= 'same')(input)
    x5=Conv2D(filters = x5_out,strides=strides, kernel_size= (5,5), padding= 'same')(x5)
    x5=relu(x5)

    if pooling_option == 'max':
        x_po = MaxPooling2D(pool_size=(3,3), strides = pool_strides, padding="same")(input)
        if pool_output == 0:
            pass
        else:
            x_po = Conv2D(filters = pool_output, kernel_size= (1,1), padding= 'same')(x_po)
    elif pooling_option == 'l2':
        x_po = l2_norm2d(input)
        x_po = Conv2D(filters = pool_output, kernel_size= (1,1), padding= 'same')(x_po)

    if input_size==0:
        result=tf.concat([x3,x5,x_po],axis=3)
    else:
        result =  tf.concat([x1,x3,x5,x_po],axis=3)
    return result

In [ ]:
# orginal
input_shape= (224,224,3)
relu=tf.keras.layers.ReLU(max_value=None, negative_slope=0.0, threshold=0.0)


input=Input(shape=input_shape)
# conv,max pool + norm
x1= Conv2D(filters = 64 ,strides=(2,2), kernel_size= (7,7), padding= 'same')(input)
x1=MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="same")(x1)
x1=BatchNormalization(momentum=0.99,epsilon=0.001)(x1)
x1=relu(x1)
# inception (2), norm + max pool
x2= Conv2D(filters = 64, kernel_size= (1,1), padding= 'same')(x1)
x2=Conv2D(filters = 192,strides=(1,1), kernel_size= (3,3), padding= 'same')(x2)
x2=relu(x2)
x2=BatchNormalization(momentum=0.99,epsilon=0.001)(x2)
x2=MaxPooling2D(pool_size=(3, 3), strides=(2,2), padding="same")(x2)
#inception (3a) 
x3 = inception(input=x2,input_size=64,x3_in=96,x3_out=128,x5_in=16,x5_out=32,pooling_option='max',pool_output=32)
#inception (3b) 
x4 = inception(input=x3,input_size=64,x3_in=96,x3_out=128,x5_in=32,x5_out=64,pooling_option='l2',pool_output=64)
#inception (3c)
x5 = inception(input=x4,input_size=0,x3_in=128,x3_out=256,x5_in=32,x5_out=64,pooling_option='max',pool_output=0, strides=(2,2),pool_strides=(2,2))
#inception (4a)
x6=inception(input=x5,input_size=256,x3_in=96,x3_out=192,x5_in=32,x5_out=64,pooling_option='l2',pool_output=128)
#inception (4b)
x7=inception(input=x6,input_size=224,x3_in=112,x3_out=224,x5_in=32,x5_out=64,pooling_option='l2',pool_output=128)
# inception (4c)
x8=inception(input=x7,input_size=192,x3_in=128,x3_out=256,x5_in=32,x5_out=64,pooling_option='l2',pool_output=128)
#inception (4d) 
x9=inception(input=x8,input_size=160,x3_in=144,x3_out=288,x5_in=32,x5_out=64,pooling_option='l2',pool_output=128)
#inception (4e) 
x10 = inception(input=x9,input_size=0,x3_in=160,x3_out=256,x5_in=64,x5_out=128,pooling_option='max',pool_output=0, strides=(2,2),pool_strides=(2,2))
#inception (5a)
x11 = inception(input=x10,input_size=384,x3_in=192,x3_out=384,x5_in=48,x5_out=128,pooling_option='l2',pool_output=128)
#inception (5b)
x12=inception(input=x11,input_size=384,x3_in=192,x3_out=384,x5_in=48,x5_out=128,pooling_option='max',pool_output=128)
# avg pool
x13= AveragePooling2D(pool_size=(7,7), strides=(1,1))(x12)
x13= Flatten()(x13)
#fully conn
x14 = Dense(128)(x13)
#L2 normalization
x15=tf.math.l2_normalize(x14, axis=None, epsilon=1e-12, name='L2 normalization')

NN2 = Model(input, [x15], name='nn2')

# model.summary()

In [ ]:
NN2.load_weights('/content/drive/MyDrive/facenet/NN2_2.h5')

In [ ]:
model.save_weights('/content/drive/MyDrive/facenet/NN2.h5')

In [ ]:
def scheduler(epoch):
    lr = 1e-6
    if epoch <1100:
        return lr/(1+0.01*epoch)
    else:
        return lr/(1+0.005*(epoch-500))

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)


BATCH_SIZE = 256
EPOCH =2000
filename='/content/drive/MyDrive/facenet/NN2_2.h5'

optimizer = tf.keras.optimizers.Adam(lr=0.005,epsilon=1e-7)
# optimizer=tf.keras.optimizers.SGD(
#     learning_rate=0.01, momentum=0.09
# )

NN2.compile(optimizer=optimizer,
              loss=get_loss_function(margin=1,All_positive=False,semi=True)
              )

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


checkpoint = ModelCheckpoint(filename,             
                             verbose=1,            
                             save_best_only=True,  
                             save_weights_only=True
                            
                            )



NN2.fit(x=x_train, 
          y=y_train, 
          epochs=EPOCH,
          batch_size=BATCH_SIZE, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback,callback,checkpoint])
        # callbacks=[tensorboard_callback,callback])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/2000
40/40 [==============================] - ETA: 0s - loss: 0.6552
Epoch 1: val_loss improved from inf to 0.59113, saving model to /content/drive/MyDrive/facenet/NN2_2.h5
40/40 [==============================] - 36s 700ms/step - loss: 0.6552 - val_loss: 0.5911 - lr: 1.0000e-06
Epoch 2/2000
40/40 [==============================] - ETA: 0s - loss: 0.6695
Epoch 2: val_loss did not improve from 0.59113
40/40 [==============================] - 24s 598ms/step - loss: 0.6695 - val_loss: 0.5917 - lr: 9.9010e-07
Epoch 3/2000
40/40 [==============================] - ETA: 0s - loss: 0.6639
Epoch 3: val_loss did not improve from 0.59113
40/40 [==============================] - 24s 597ms/step - loss: 0.6639 - val_loss: 0.5931 - lr: 9.8039e-07
Epoch 4/2000
40/40 [==============================] - ETA: 0s - loss: 0.6429
Epoch 4: val_loss did not improve from 0.59113
40/40 [==============================] - 24s 599ms/step - loss: 0.6429 - val_loss: 0.5927 - lr: 9.7087e-07
Epoch 5/2000
40/40 

KeyboardInterrupt: ignored

In [ ]:
import io
# Evaluate the network
results = NN2.predict(x_test)
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
[out_m.write(str(x) + "\n") for x in np.array(y_test)]
out_m.close()


try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
for i in range(20):
    plt.imshow(x_test[i])
    plt.show()